Let's make that last visualization look nicer. We're still using the [Aerial Waterfowl Survey Data](https://data.delaware.gov/Energy-and-Environment/Aerial-Waterfowl-Survey-Data/bxyv-7mgn). 


In [ ]:
# Import our libraries
import pandas as pd

import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode()


In [ ]:
# Run the transformations from before to make sure everyone is 
# at the same place.
url = "https://data.delaware.gov/api/views/bxyv-7mgn/rows.csv?accessType=DOWNLOAD"
waterfowl_df = pd.read_csv(url)
waterfowl_df_january = waterfowl_df[waterfowl_df['Month']=='January']
waterfowl_df_january_sub = waterfowl_df_january[waterfowl_df_january['Time Period']!='Late']

waterfowl_df_january_sub_by_year = waterfowl_df_january_sub.groupby('Year').sum()

years = [str(year) for year in waterfowl_df_january_sub_by_year.index]


In [ ]:
# Now let's plot the top three.
# This is just like in the last chart, but options to go.Scatter each have their own line

bird_names = ['Canada Goose', 'American Black Duck', 'Mallard']

data = []

for bird_name in bird_names:
    
    single_bird = waterfowl_df_january_sub[['Year', bird_name]].groupby('Year').sum()

    bird_counts = [int(total) for total in single_bird[bird_name]]
    
    # Cheat and re-usse the years variable from before
    data.append(
        go.Scatter(
            x=years, 
            y=bird_counts, 
            mode="markers+lines", 
            name=bird_name
        )
    )

layout=go.Layout(title="Top three birds", xaxis={'title':'Year'}, yaxis={'title':'Number counted'})

figure=go.Figure(data=data,layout=layout)

offline.iplot(figure, filename='top_three')

Before diving into Plotly's documentation, is there anything in the block above that you want to edit and test?

How can you make the Title more descriptive? 

How can you edit the order of the birds in the legend on the right of the graph?

Notice how mode is set to "markers+lines"? What happens if you make it just "markers" or "lines"?

How would you change the hover text? Seeing "123.431K" looks weird (Canada Goose, 2001)



To see what options there are to make Plotly look nicer, checkout [the documentation for Scatter](https://plot.ly/python/line-and-scatter/). 

"Style" is inherently subjective, but to get a feel for best practices consult a style guide. Personally I like the [Sunlight Foundation's](https://github.com/amycesal/dataviz-style-guide/blob/master/Sunlight-StyleGuide-DataViz.pdf), but it's a PDF. This one by the [Uban Institute](http://urbaninstitute.github.io/graphics-styleguide/) is a good one to read in a browser.

Note: We're jumping ahead to just the variables for Scatter. To get a general overview of ploty, read the [user guide](https://plot.ly/python/user-guide/).

Click on the "+" sign, then copy and paste the charting code. Make some edits and play around with it!

In [ ]:
# Here's what I made
# I changed the hover text to be more descriptive
# The title is programatically changed to be show the bird names from the list, as well as the year range


# Try shape options: "linear" | "spline" | "hv" | "vh" | "hvh" | "vhv"

bird_names = ['Canada Goose', 'American Black Duck', 'Mallard']

data = []

for bird_name in bird_names:
    
    single_bird = waterfowl_df_january_sub[['Year', bird_name]].groupby('Year').sum()

    bird_counts = [int(total) for total in single_bird[bird_name]]
    
    # Cheat and re-usse the years variable from before
    data.append(
        go.Scatter(
            x=years, 
            y=bird_counts, 
            mode="markers+lines", 
            name=bird_name,
            hoverinfo="text",
            hovertext=['{:,} {} in {}'.format(bird_counts[i], bird_name, years[i]) for i in range(len(bird_counts))],
            line={'shape':'spline', 'smoothing':0.5, 'width':3}, # try adding , 'smoothing':0.5, 'width':3
            marker={'size': 6, 'opacity': .7}
        )
    )

layout=go.Layout(
    title='{} from {} to {}'.format(', '.join(bird_names), years[0], years[-1]), 
    xaxis={'title':'Year'}, 
    yaxis={'title':'Number counted'}
)

figure=go.Figure(data=data,layout=layout)

offline.iplot(figure, filename='top_three')